<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/Sequence_to_Sequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import zipfile
import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import files
uploaded=files.upload()

Saving fra.txt to fra.txt


In [3]:
lines=pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print("전체 샘플의 개수 : ", len(lines))

전체 샘플의 개수 :  227815


In [4]:
lines.head()

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [5]:
lines=lines.loc[:, 'src':'tar']
lines=lines[0:60000]#6만개만 사용

In [6]:
lines['tar']=lines['tar'].apply(lambda x:'\t'+x+'\n') #시작심볼 \t와 종료심볼 \n을 추가한다.
lines

,src,tar
0,Go.,\tVa !\n
1,Go.,\tMarche.\n
2,Go.,\tEn route !\n
3,Go.,\tBouge !\n
4,Hi.,\tSalut !\n
...,...,...
59995,These are my students.,\tCe sont mes élèves.\n
59996,These aren't my ideas.,\tCe ne sont pas mes idées.\n
59997,These books are Tom's.,\tCes livres sont à Tom.\n
59998,These books are heavy.,\tCes livres sont lourds.\n


In [7]:
#문자 집합 구축
src_vocab=set()
for line in lines['src']:
  for char in line:
    src_vocab.add(char)

tar_vocab=set()
for line in lines['tar']:
  for char in line:
    tar_vocab.add(char)


In [8]:
src_vocab_size=len(src_vocab)+1
tar_vocab_size=len(tar_vocab)+1
print("source 문장의 char 집합 : ", src_vocab_size)
print("target 문장의 char 집합 : ", tar_vocab_size)

source 문장의 char 집합 :  80
target 문장의 char 집합 :  104


In [9]:
src_vocab=sorted(list(src_vocab))
tar_vocab=sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [10]:
src_to_index=dict([(word, i+1)for i, word in enumerate(src_vocab)])
tar_to_index=dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [11]:
encoder_input=[]
for line in lines['src']:
  encoded_line=[]
  for char in line:
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print("source 문장의 정수 인코딩 : ", encoder_input[:5])

source 문장의 정수 인코딩 :  [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [12]:
decoder_input=[]
for line in lines['tar']:
  encoded_line=[]
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print("target 문장의 정수 인코딩 : ", decoder_input[:5])

target 문장의 정수 인코딩 :  [[1, 48, 52, 3, 4, 2], [1, 39, 52, 69, 54, 59, 56, 14, 2], [1, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 2], [1, 28, 66, 72, 58, 56, 3, 4, 2], [1, 45, 52, 63, 72, 71, 3, 4, 2]]


In [13]:
decoder_target=[] #예측값과는 다르게 실제값에서는 sos가 필요가 없음
for line in lines['tar']:
  timestep=0
  encoded_line=[]
  for char in line:
    if timestep>0:
      encoded_line.append(tar_to_index[char]) #/t를 뺀다.
    timestep+=1
  decoder_target.append(encoded_line)
print("target 문장 레이블의 정수 인코딩 : ", decoder_target[:5])

target 문장 레이블의 정수 인코딩 :  [[48, 52, 3, 4, 2], [39, 52, 69, 54, 59, 56, 14, 2], [31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 2], [28, 66, 72, 58, 56, 3, 4, 2], [45, 52, 63, 72, 71, 3, 4, 2]]


In [14]:
max_src_len=max([len(line)for line in lines['src']])
max_tar_len=(max([len(line) for line in lines['tar']]))
print('source 문장의 최대 길이 : ', max_src_len)
print('target 문장의 최대 길이 : ', max_tar_len)

source 문장의 최대 길이 :  22
target 문장의 최대 길이 :  74


In [15]:
#source와 target은 왠만하면 길이가 다 다르므로 각각 패딩을 한다.
encoder_input=pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input=pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target=pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [16]:
encoder_input=to_categorical(encoder_input)
decoder_input=to_categorical(decoder_input)
decoder_target=to_categorical(decoder_target)

In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np


In [18]:
encoder_inputs=Input(shape=(None, src_vocab_size)) #src_vocab_size만큼의 특성을 가지고 알 수 없는 시점을 가지고 들어간다.(None).
encoder_lstm=LSTM(units=256, return_state=True) #units=hidden_units, return_state=True이면 내부 상태를 반환한다.
encoder_output, state_h, state_c=encoder_lstm(encoder_inputs)#encoder_inputs와 연결하고 내부 상태를 state_h와 state_c에 저장한다.
encoder_states=[state_h,state_c] #은닉상태(hidden)와 셀(cell)상태 모두 디코더로 전달하기 위해 컨텍스트 벡터로 만든다.

In [19]:
decoder_inputs=Input(shape=(None, tar_vocab_size))
#decoder에도 input이 있는 이유는 교사강요 : 훈련과정에서 이전 시점의 예측값이 아닌 이전 시점의 실제값을 훈련시킨다.
decoder_lstm=LSTM(units=256, return_sequences=True, return_state=True) #디코더의 은닉상태 크기도 인코더와 똑같이 해준다.

#decoder에게 encoder의 은닉상태와 셀상태 전달
decoder_outputs, _, _=decoder_lstm(decoder_inputs, initial_state=encoder_states) #decoder_inputs와 연결하고 encoder의 층과 같은 상태를 위해 initial_state으로 state_h와 state_c(컨텍스트 벡터)를 가져와서 lstm에 저장한다. -> encoder와 decoder 레이어의 단위 크기가 같아야 한다.
#훈련 과정에서는 디코더의 은닉상태와 셀상태를 리턴하지 않기 때문에 변수를 따로 지정하지 않음.
decoder_softmax_layer=Dense(tar_vocab_size, activation='softmax')
decoder_outputs=decoder_softmax_layer(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs) #입력은 encoder_inputs와 decoder_inputs의 병렬이고 출력은 decoder_outputs이다.
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [20]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 461s 610ms/step - loss: 0.8580 - val_loss: 0.7870
Epoch 2/40
750/750 [==============================] - 393s 524ms/step - loss: 0.5904 - val_loss: 0.6882
Epoch 3/40
750/750 [==============================] - 423s 565ms/step - loss: 0.5180 - val_loss: 0.6173
Epoch 4/40
750/750 [==============================] - 392s 523ms/step - loss: 0.4698 - val_loss: 0.5651
Epoch 5/40
750/750 [==============================] - 403s 537ms/step - loss: 0.4311 - val_loss: 0.5349
Epoch 6/40
750/750 [==============================] - 445s 594ms/step - loss: 0.4033 - val_loss: 0.5016
Epoch 7/40
750/750 [==============================] - 446s 595ms/step - loss: 0.3819 - val_loss: 0.4815
Epoch 8/40
750/750 [==============================] - 411s 548ms/step - loss: 0.3647 - val_loss: 0.4646
Epoch 9/40
750/750 [==============================] - 389s 519ms/step - loss: 0.3505 - val_loss: 0.4505
Epoch 10/40
750/750 [==============================] - 438s 585m

In [22]:
#동작할 때는 디코더의 은닉 상태와 셀 상태가 필요하다
encoder_model=Model(inputs=encoder_inputs, outputs=encoder_states)

In [23]:
#이전 시점의 상태들을 저장할 텐서
decoder_state_input_h=Input(shape=(256,))
decoder_state_input_c=Input(shape=(256,))
decoder_state_inputs=[decoder_state_input_h, decoder_state_input_c]

#문장의 다음 단어 예측을 위해 초기 상태(initial_state)를 이전 시점의 상태로 사용
decoder_outputs, state_h, state_c=decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

#훈련 과정과는 다르게 LSTM이 리턴하는 은닉상태와 셀 상태를 버리지 않음
decoder_states=[state_h, state_c]
decoder_outputs=decoder_softmax_layer(decoder_outputs)
decoder_model=Model(inputs=[decoder_inputs]+decoder_state_inputs, outputs=[decoder_outputs]+decoder_states)


In [24]:
index_to_src=dict((i, char) for char, i in src_to_index.items())
index_to_tar=dict((i, char)for char, i in tar_to_index.items())

In [25]:
def decode_sequence(input_seq):
  #입력한 후 인코더의 상태를 얻음
  states_value=encoder_model.predict(input_seq)

  #<SOS>에 해당하는 원-핫 벡터 생성 -> \t의 원핫벡터만 따로 생성
  target_seq=np.zeros((1,1,tar_vocab_size))
  target_seq[0,0,tar_to_index['\t']]=1.

  stop_condition=False
  decoded_sentence=""

  #stop_condition이 True가 될때까지 루프 반복
  while not stop_condition:
    #이전 시점의 상태 states_value(encoder의 은닉상태와 셀상태)를 현 시점의 초기 상태로 사용
    output_tokens, h, c=decoder_model.predict([target_seq]+states_value) #이전 시점의 예측 결과와 인코더의 이전 시점의 상태로 디코더의 예측과 은닉상태, 셀상태를 저장.

    #예측 결과를 문자로 변환
    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    sampled_char=index_to_tar[sampled_token_index]

    #현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence+=sampled_char

    #<eos>에 도달하거나 최대 길이를 넘으면 중단
    if(sampled_char=='\n' or len(decoded_sentence)>max_tar_len):
      stop_condition=True

    #현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq=np.zeros((1,1,tar_vocab_size))
    target_seq[0,0,sampled_token_index]=1.

    #현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value=[h,c]

  return decoded_sentence

In [27]:
for seq_index in [3,50,100,300,1001]:
  input_seq=encoder_input[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print(35*"-")
  print('입력 문장 : ', lines.src[seq_index])
  print('정답 문장 : ', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장 : ', decoded_sentence[1:len(decoded_sentence)-1]) #'\n'을 빼고 출력


1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장 :  Go.
정답 문장 :  ouge !
번역 문장 :  ontinue.
1/1 [==============================] - 0s 34ms/step
-----------------------------------
입력 문장 :  Hello!
정답 문장 :  onjour !
번역 문장 :  ucun avec moi.
1/1 [==============================] - 0s 26ms/step
-----------------------------------
입력 문장 :  Got it!
정답 문장 :  'ai pigé !
번역 문장 :  étends !
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장 :  Go home.
정답 문장 :  entre à la maison.
번역 문장 :  a te prendre un peu de temps !
1/1 [==============================] - 0s 27ms/step
-----------------------------------
입력 문장 :  Get going.
정답 문장 :  n avant.
번역 문장 :  éparez-vous.
